# Deep Learning from scratch
## Convolutional Neural Networks for cat classifer

In [2]:
from os import listdir
from os.path import isfile, join
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

%matplotlib inline

cat_dir = 'cat/'
not_cat_dir = 'not-cat/'
wolf_dir = 'wolf/'

cat_files = [f for f in listdir(cat_dir) if isfile(join(cat_dir, f))]
not_cat_files = [f for f in listdir(not_cat_dir) if isfile(join(not_cat_dir, f))]
wolf_files = [f for f in listdir(wolf_dir) if isfile(join(wolf_dir, f))]

In [3]:
width = 128
height = 128

In [7]:
x_cat = []
for filename in cat_files:
    with Image.open(cat_dir+filename) as im:
        if im.mode == 'RGB':
            nim = im.resize((width, height), Image.BILINEAR)
            pixel_values = np.array(nim.getdata()).reshape((width, height, 3))
            x_cat.append(pixel_values)
x_cat_array = np.array(x_cat)
print(x_cat_array.shape)
#plt.imshow(x_cat_array[0,:])

(1482, 128, 128, 3)


In [8]:
x_not_cat = []
for filename in not_cat_files:
    with Image.open(not_cat_dir+filename) as im:
        if im.mode == 'RGB':
            nim = im.resize((width, height), Image.BILINEAR)
            pixel_values = np.array(nim.getdata()).reshape((width, height, 3))
            x_not_cat.append(pixel_values)
x_not_cat_array = np.array(x_not_cat)
print(x_not_cat_array.shape)
#plt.imshow(x_not_cat_array[0,:])

(1404, 128, 128, 3)


In [9]:
x_wolf = []
for filename in wolf_files:
    with Image.open(wolf_dir+filename) as im:
        if im.mode == 'RGB':
            nim = im.resize((width, height), Image.BILINEAR)
            pixel_values = np.array(nim.getdata()).reshape((width, height, 3))
            x_wolf.append(pixel_values)
x_wolf_array = np.array(x_wolf)
print(x_wolf_array.shape)
#plt.imshow(x_wolf_array[0,:])

(1390, 128, 128, 3)


In [10]:
y_not_cat_array = np.zeros((x_not_cat_array.shape[0]+x_wolf_array.shape[0], 1))
y_cat_array = np.ones((x_cat_array.shape[0], 1))

y_array = np.append(y_not_cat_array, y_cat_array, axis=0)
print(y_array.shape)

(4276, 1)


In [11]:
x_other_array = np.append(x_not_cat_array, x_wolf_array, axis=0)

x_array = np.append(x_other_array, x_cat_array, axis=0)
print(x_array.shape)

(4276, 128, 128, 3)


In [14]:
m_test = np.rint(x_array.shape[0]*0.2)
m_dev = m_test
m_train = x_array.shape[0]-m_test-m_dev
print("Number of training examples: " + str(m_train))
print("Number of developing examples: " + str(m_dev))
print("Number of testing examples: " + str(m_test))

Number of training examples: 2566.0
Number of developing examples: 855.0
Number of testing examples: 855.0
